In [1]:
import os

# Define the new working directory relative to the current working directory
new_working_directory = os.path.join(os.getcwd(), '..', '..', '..') # Set to MLAgentBenhc
print("New working directory: ", new_working_directory)

# Change the current working directory
os.chdir(new_working_directory)
print("New Working Directory:", os.getcwd()) # Should be ...\MLAgentBench

New working directory:  c:\Users\kevihuang\OneDrive - Microsoft\Desktop\projects\MLAgentBench\MLAgentBench_v2\agents\Debug\..\..\..
New Working Directory: c:\Users\kevihuang\OneDrive - Microsoft\Desktop\projects\MLAgentBench


In [2]:
# General Environment and Agent imports
from MLAgentBench_v2.agents.agent import Agent
from types import SimpleNamespace
from MLAgentBench_v2.environment import Environment
from datetime import date

# Agent specific imports

No module named 'helm'
Could not load CRFM API key crfm_api_key.txt.
[Errno 2] No such file or directory: 'claude_api_key.txt'
Could not load anthropic API key claude_api_key.txt.
[Errno 2] No such file or directory: 'openai_api_key.txt'
Could not load OpenAI API key openai_api_key.txt.


In [3]:
# Instantiate an environment
TASK = 'home-data-for-ml-course'
MODEL = 'gpt-3.5-turbo-0125'
AGENT_NAME = 'eureka-v2'
VERSION = '0'
RESEARCH_PROBLEM = '''''' # Manually define the goal for the agent based on Kaggle competition description

env = Environment(SimpleNamespace(
    task=TASK,
    task_type='kaggle',
    log_dir=f'logs/{date.today().strftime("%Y-%m-%d")}__{TASK}__{AGENT_NAME}__{MODEL}__v{VERSION}', # date__task__agent__llm__version
    work_dir='workspace',
    llm_name=MODEL,
    research_problem=RESEARCH_PROBLEM
))

Initializing environment...
args namespace(task='home-data-for-ml-course', task_type='kaggle', log_dir='logs/2024-05-05__home-data-for-ml-course__debug__gpt-3.5-turbo-0125__v0', work_dir='workspace', llm_name='gpt-3.5-turbo-0125')
Preparing task home-data-for-ml-course , of type:  kaggle


In [3]:
# Instantiate the agent
class DebugAgent(Agent):
    def __init__(self, env):
        super().__init__(env)
        self.env = env # allows you to interact with the environment using debug_agent.env

debug_agent = DebugAgent(env)

In [5]:
# Write the script manually

# Grabbing code that I know executes

python_code = '''import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

# Load the data
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

# Remove rows with missing target
train_data.dropna(axis=0, subset=['SalePrice'], inplace=True)

# Separate target from predictors
y = train_data.SalePrice
X = train_data.drop(['SalePrice'], axis=1)

# Divide data into training and validation subsets
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

# Categorical columns with relatively low cardinality (convenient but arbitrary limit set at 10)
categorical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and X_train_full[cname].dtype == "object"]

# Numerical columns
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Columns that will be considered for the model
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = test_data[my_cols].copy()

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define the model
model = XGBRegressor(n_estimators=1000, learning_rate=0.05)

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# Preprocessing of training data, fit model
my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds_valid = my_pipeline.predict(X_valid)

# Evaluate the model
mae = mean_absolute_error(y_valid, preds_valid)
print("Validation MAE:", mae)

# Check if MAE is less than 15,000
if mae < 15000:
    # Preprocessing of test data, fit model
    preds_test = my_pipeline.predict(X_test)
    
    # Save test predictions to file
    output = pd.DataFrame({'Id': X_test.Id,
                           'SalePrice': preds_test})
    output.to_csv('submission.csv', index=False)
    print("Test predictions saved to submission.csv")
else:
    print("Validation MAE is not less than 15000. Consider revising the model.")
'''

write_args = {
    'file_name': f'train.py',
    'content': python_code,
    'update_files_action_result_history': False,
}
debug_agent.available_actions['writeFile'](**write_args)



--- LOGGING NEW ACTION ---
Step: 5
Calling function wrapped_write_file(args = (), kwargs = {'file_name': 'train.py', 'content': 'import pandas as pd\nfrom sklearn.model_selection import train_test_split\nfrom sklearn.metrics import mean_absolute_error\nfrom xgboost import XGBRegressor\nfrom sklearn.impute import SimpleImputer\nfrom sklearn.compose import ColumnTransformer\nfrom sklearn.pipeline import Pipeline\nfrom sklearn.preprocessing import OneHotEncoder\n\n# Load the data\ntrain_data = pd.read_csv(\'train.csv\')\ntest_data = pd.read_csv(\'test.csv\')\n\n# Remove rows with missing target\ntrain_data.dropna(axis=0, subset=[\'SalePrice\'], inplace=True)\n\n# Separate target from predictors\ny = train_data.SalePrice\nX = train_data.drop([\'SalePrice\'], axis=1)\n\n# Divide data into training and validation subsets\nX_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)\n\n# Categorical columns with relatively low cardinal

'File train.py written successfully.'

In [6]:
# Execute the script
execute_args = {
    'script_name': 'train.py',
    'update_files_action_result_history': False,
}
result = debug_agent.env.execute_script(**execute_args)



--- LOGGING NEW ACTION ---
Step: 6
Calling function wrapped_execute_script(args = (), kwargs = {'script_name': 'train.py', 'work_dir': 'workspace\\home-data-for-ml-course_branch'})



Script output: Validation MAE: 17007.92463077911
Validation MAE is not less than 15000. Consider revising the model.



--- LOGGING NEW ACTION ---
Step: 6
Calling function wrapped_complete_text_openai(args = (), kwargs = {'system_prompt': 'You are a helpful assistant. Your goal is to check if the code outputs the train and validation score, specifically not from log values, but normal values, and make sure that the code for calculating validation score is actually from a validation set. If so, then extract the train and validation score value from the result. If the code doesn\'t output the train and validation score or its not for normal values or the code doesn\'t actually calculate and print validation score from a validation set, then please write "inf" as the traing and validation score. Please also write down what type of validation score it is (ex. Mean Absolute Error (MAE), Root Mean Squared Error (RMSE), accuracy, etc.)\n            \n            Example:\n            ```json\

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>